## Concept traversal bugfixing

In [1]:
import os
import lxml.etree as ET
import numpy as np
from pynxtools.nexus import nexus

fpath="/home/kaiobach/Research/hu_hu_hu/sprint19/em/.py3.11.5/lib64/python3.11/site-packages/pynxtools/definitions/contributed_definitions"
print(os.getcwd())
nxdl_file_path = f"{fpath}/NXem.nxdl.xml"
elem = ET.parse(nxdl_file_path).getroot()
# read_nexus -d gives concept_path + class_path + documentation for all its superclasses as well
# -c requires the concept_path with a leading /NXmyfancyappdefname/....
# test_nxdl_concept_path = "NXem:/ENTRY/ROI/ebsd/indexing/phaseID/ipfID/map"
test_nxdl_concept_path = "/ENTRY/ROI/ebsd/indexing/phaseID/ipfID/map/data"
# as defined in NXDL in general but not a concept name we go beyond the base classes (you cannot find such valid path directly in the NeXus glossary because if you dgo tb bsclass you havent spelled it out base class was same as reference, inferring that what is in the base class we can compose to what is)
test_nx_data_path = "/entry/roi/ebsd/indexing/phase1/ipf1/map/data"  # aka hdf_path
# actual name how an instance of an nxdl_concept_path shows up in HDF5
test_nx_template_path = "/ENTRY[entry]/ROI[roi]/ebsd/indexing/phaseID[phase1]/ipfID[ipf1]/map/data"
# pynxtools-specific guidance to people working with the template entry one assumption made:
# either we write a fixed name or a variable name
# (two kinds: uppercase in name or the property nameType = any is specified in the NXDL)
# possibility to omit ID completely
test_class_path = "/NXentry/NXroi/NXem_ebsd/NXprocess/NXcrystal_structure/NXms_ipf/NXdata/NX_NUMBER"

n_len = len(test_nxdl_concept_path.split("/"))
for pos in np.arange(2, n_len + 1):
    trg = f"/{'/'.join(test_nxdl_concept_path.split('/')[1:pos])}"
    print(trg)
    node = nexus.get_node_at_nxdl_path(trg, elem=elem)
    # get_node_at_nxdl_path with just get_node_at_nxdl_path("/", elem=elem) starting from a root
    # an application definition is a possibility for defining a specialized NXentry
    print(node.attrib)

# node = nexus.get_node_at_nxdl_path("/ENTRY/ROI/ebsd", elem=elem) # inspect nxdlpath still all the way from root
# node = nexus.get_node_at_nxdl_path("/ENTRY/ROI/ebsd/indexing", elem=elem) # oops, starting again as if root nxdlpath="/indexing" where nxdlpath="/ENTRY/ROI/ebsd/indexing" is expected
# node = nexus.get_node_at_nxdl_path("/ENTRY/ROI/ebsd/indexing/roi", elem=elem) # oops, starting again as if root ... /indexing but should be /ENTRY/ROI/ebsd/indexing
# node = nexus.get_node_at_nxdl_path("/ENTRY/ROI/ebsd/indexing/phaseID", elem=elem)  # walked fine further with however /indexing as root
# node = nexus.get_node_at_nxdl_path("/ENTRY/ROI/ebsd/indexing/phaseID/ipfID", elem=elem)  # walked fine further with boom! again a jump /ipfID the new root
# in all cases though the code was able to resolve that the concept were but after the root was redefined only!
# node = nexus.get_node_at_nxdl_path("/ENTRY/ROI/ebsd/indexing/phaseID/ipfID/map", elem=elem) # boom ! yet another root instead of map that is an NXdata within an NXms_ipf
# print(node.attrib)

/home/kaiobach/Research/hu_hu_hu/sprint19/em/pynxtools_em
/ENTRY
{'type': 'NXentry', 'minOccurs': '1', 'maxOccurs': 'unbounded', 'nxdlbase': '', 'nxdlbase_class': 'application', 'nxdlpath': '/ENTRY'}
/ENTRY/ROI
{'type': 'NXroi', 'minOccurs': '0', 'maxOccurs': 'unbounded', 'nxdlbase': '', 'nxdlbase_class': 'application', 'nxdlpath': '/ENTRY/ROI'}
/ENTRY/ROI/ebsd
{'name': 'ebsd', 'type': 'NXem_ebsd', 'optional': 'true', 'nxdlbase': '', 'nxdlbase_class': 'application', 'nxdlpath': '/ENTRY/ROI/ebsd'}
/ENTRY/ROI/ebsd/indexing
{'name': 'indexing', 'type': 'NXprocess', 'nxdlbase': '/home/kaiobach/Research/hu_hu_hu/sprint19/em/.py3.11.5/lib/python3.11/site-packages/pynxtools/definitions/contributed_definitions/NXem_ebsd.nxdl.xml', 'nxdlbase_class': 'base', 'nxdlpath': '/indexing'}
/ENTRY/ROI/ebsd/indexing/phaseID
{'name': 'phaseID', 'type': 'NXcrystal_structure', 'nxdlbase': '/home/kaiobach/Research/hu_hu_hu/sprint19/em/.py3.11.5/lib/python3.11/site-packages/pynxtools/definitions/contributed_d

In [ ]:
f"/{'/'.join('/ENTRY/ROI/ebsd/indexing/phaseID/ipfID/map'.split('/')[1:3])}"